# Spanish Electricity Market Analysis: Part 1/2

## Part 2: Statistics and Data Visualization

In [1]:
import pandas as pd
import seaborn as sns
import os
import matplotlib as mpl
import matplotlib.pyplot as plt
from sqlalchemy import create_engine
import datetime

sns.set()

### Daily electricity generation. Visualization

In [ ]:
os.getcwd()
PBF_total_df = pd.read_csv('cleaned_data/generation_per_source.csv') #reading the csv file with the aggregated data from 2014-2018. 

### Daily electricity generation per source. Visualization

In [3]:
os.getcwd()

'/home/ingrid/Documents/DA_Ironhack/Week5/Project-Week-5-Your-Own-Project/your-project'

In [5]:
gen_source_df = pd.read_csv('cleaned_data/generation_per_source.csv') #reading the csv file with the aggregated data from 2014-2018. 

In [6]:
#we create a multi index pd dataframe in order to facilitate the data visualization later
gen_source_df.set_index(['date', 'source'], inplace=True)
gen_source_df.sort_index(inplace=True)
gen_source_df

value_MWh
date       source                
2014-01-01 CHP             4497.5
           coal            2498.7
           hydro_1        87564.8
           hydro_2        15226.2
           natural_gas    75993.1
           nuclear       144654.6
           solar_PV        7027.3
           wind_power    277443.9
2014-01-02 CHP            14541.1
           coal           19741.4
           hydro_1        94432.6
           hydro_2        21979.8
           natural_gas    83437.4
           nuclear       146177.8
           solar_PV       12817.8
           wind_power    237323.1
2014-01-03 CHP             3555.6
           coal            1288.0
           hydro_1        92236.2
           hydro_2         3192.2
           natural_gas    65023.9
           nuclear       145055.3
           solar_PV       12532.5
           wind_power    329506.6
2014-01-04 CHP             2310.2
           hydro_1        94189.3
           hydro_2         4540.8
           natural_gas    64036.6
           nuclear       145200.2
           solar_PV       14108.3
...                           ...
2018-12-27 nuclear       145380.0
           solar_PV       14287.2
           wind_power    120618.6
2018-12-28 CHP              424.0
           coal            1776.0
           hydro_1        75207.5
           hydro_2         8738.8
           natural_gas    74152.7
           nuclear       145418.4
           solar_PV       15096.0
           wind_power    188485.7
2018-12-29 coal             683.0
           hydro_1        77932.7
           hydro_2         9020.0
           natural_gas    72911.5
           nuclear       145392.0
           solar_PV       18946.6
           wind_power    151707.2
2018-12-30 CHP            37286.6
           coal           34197.3
           hydro_1        85938.3
           hydro_2         8466.2
           natural_gas    66782.2
           nuclear       145430.4
           solar_PV       18812.7
           wind_power     97631.5
2018-12-31 hydro_1        71905.4
           hydro_2        10764.0
           natural_gas    61770.5
           solar_PV       19967.0

[14568 rows x 1 columns]